In [1]:
import os
from pathlib import Path

import mlflow
import pandas as pd
import yaml
from dotenv import load_dotenv

os.environ["MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR"] = "false"
load_dotenv("../.env")

TEMP_DIR = "./.tmp"

In [ ]:
Path(TEMP_DIR).mkdir(exist_ok=True, parents=True)
mlflow_uri = os.getenv("MLFLOW_TRACKING_URI", "./result/mlruns")
mlflow.set_tracking_uri(mlflow_uri)

with open("../config/__base__/config.yaml", "r") as f:
    cfg = yaml.safe_load(f)
experiment_name = cfg["mlflow"]["experiment_name"]
print(f"Experiment: {experiment_name}")

client = mlflow.tracking.MlflowClient()
experiment_id = client.get_experiment_by_name(experiment_name).experiment_id

options = dict(experiment_ids=experiment_id, filter_string="status='FINISHED")
runs = client.search_runs(**options)

In [34]:
results = []
for run in runs:
    metrics = {}
    run_name = run.info.run_name
    params = run.data.params
    metrics["model"] = params["Model"]
    metrics["loss"] = params["Loss"]
    metrics["dataset"] = params["Train_Dataset"]
    metrics["tag"] = run.data.tags.get("tag")
    if "v2" not in metrics["dataset"]:
        continue
    for k, v in run.data.metrics.items():
        if "system/" in k or "Epoch" in k or "loss" in k or "Learning" in k or "val" in k:
            continue
        elif "error_2d" in k:
            continue
        metrics[k] = v
    results.append(metrics)


In [ ]:
df = pd.DataFrame(results).sort_values(["dataset", "model"])
df["dataset"] = df["dataset"].replace("model", "モデル")
df

In [ ]:
df = df.drop(['model', 'loss', 'fps_test', 'Speed/ms_test'], axis=1)
df

In [ ]:
print(df.to_markdown(index=False))